Вариант 4
(Методы):
1)замена критериев ограничениями;
2)формирование и сужение множества Парето;
3)взвешивание и объединение критериев;
4)анализ иерархий

Входные данные

In [205]:
from plotly.graph_objects import Figure, Table, Scatter
from plotly.offline import iplot
from plotly import express as px
import numpy as np
from scipy.special import softmax
from plotly.subplots import make_subplots

In [206]:
# Альтернативы
alternatives = ['Автострада', 'Шоссе', 'Грунтовка', 'Проселок']

# Критерии
criterions = ['Расстояние',
              'Качество покрытия',
              'Контроль',
              'Инфраструктура']

# Для матрицы оценок результатов
distanс = [3.,4.,6.,7.] # Чем больше расстояние, тем меньше оценка
quality = [8.,6.,3.,1.]
control = [2.,3.,5.,6.] # Чем больше контроль, тем меньше оценка
infrast = [7.,8.,3.,1.]


# Веса критериев
weights = [8.,
           8.,
           2.,
           4.]
# Нормализация списка весов критериев
weights = [w / 10 for w in weights]

# Матрица оценок для альтернатив
a_matrix_orig = np.array([distanс,
            quality,
            control,
            infrast], dtype=float)
print(a_matrix_orig)

# Вывод изначальной таблицы оценок
fig_beg = Figure(data=[
    Table(
        header=dict(values=['Альтернативы\Критерии'] + criterions),
        cells=dict(
            values=[
                alternatives,
                distanс,
                quality,
                control,
                infrast
            ]),
    )])
iplot(fig_beg)

# Точность после запятой
okr = 5

[[3. 4. 6. 7.]
 [8. 6. 3. 1.]
 [2. 3. 5. 6.]
 [7. 8. 3. 1.]]


Метод замены критериев ограничениями

In [207]:
# Пусть главный критерий - Расстояние
main_critrerion = 'Расстояние'
# Метки приемлемости альтернативы
alternatives_use = [True] * len(alternatives)

# Нормировка матриц оценок (кроме столбца главного критерия)
a_matrix = np.copy(a_matrix_orig)
# Цикл по столбикам (критериям)
for j, cr in enumerate(criterions):
    # Если это главный критерий, то пропустить
    if cr == main_critrerion:
        continue

    lev = 0.0
    # Минимально допустимые уровни для остальных критериев
    match(cr):
        case 'Качество покрытия':
            lev = 0.3
        case 'Контроль':
            lev = 0.1
        case 'Инфраструктура':
            lev = 0.2

    min_val = min(a_matrix[j])
    max_val = max(a_matrix[j])

    # Цикл по строчкам (значения критериев альтернатив)
    for i, al in enumerate(alternatives):
        a_matrix[j][i] = round((a_matrix[j][i] - min_val) / (max_val - min_val), okr)
        # Оставить только приемлемые значения
        if a_matrix[j][i] < lev:
            a_matrix[j][i] = 0.0
            # Отметить альтернативу, как неприемлемую
            alternatives_use[i] = False
# Вывод таблицы
fig = Figure(data=[
    Table(
        header=dict(values=['Альтернативы\Критерии'] + criterions),
        cells=dict(
            values=[
                alternatives,
                a_matrix[0],
                a_matrix[1],
                a_matrix[2],
                a_matrix[3]
            ]),
    )])
iplot(fig)

# Среди приемлемых альтернатив оставить то, у которого значение главного критерия больше
answer = ''
# цикл по индексам значений главного критерия, отсортированных по убыванию
# distanс, т.к. главный критерий - Расстояние
for i in np.argsort(distanс)[::-1]:
    # Первая допустимая альтернатива
    if alternatives_use[i]:
        answer = alternatives[i]
        break
print(answer)


Шоссе


Метод Формирования и сужения множества Парето

In [208]:
# Два главных критерия: "Расстояние" и "Инфраструктура"

# Формирование множества Парето графическим методом

# Оба критерия максимизируются, поэтому точка утопии находится в правом верхнем углу графика
utopia = [max(distanс), max(infrast)]
fig = Figure()
fig.add_trace(Scatter(x=[utopia[0]], y=[utopia[1]], text=['Точка утопии'], textposition='top center', mode='markers+text'))
fig.add_trace(Scatter(x=distanс, y=infrast, textposition='top center', text=alternatives,
                      mode='lines+markers+text', name=''))
fig.update_layout(showlegend=False, xaxis_title="Оценка расстояния", yaxis_title="Оценка инфраструктуры",)
fig.show()

# Так как 4 вариант, то 4 / 3 = 1 и остаток 1 => Евклидово расстояние
# Поиск Евклидовых расстояних от всех точке до точки утопии
evklid_rast = []
for i in range(len(alternatives)):
    x1 = distanс[i]
    y1 = infrast[i]
    x2 = utopia[0]
    y2 = utopia[1]

    d = round(((x2 - x1)**2 + (y2 - y1)**2)**0.5, okr)
    evklid_rast.append(d)

# Выбор минимального расстояния
print(evklid_rast)
answer = alternatives[np.argmin(evklid_rast)]
print(answer)

[4.12311, 3.0, 5.09902, 7.0]
Шоссе


Метода взвешивания и объединения критериев

In [209]:
# Нормализация матрицы оценок (все столбцы)
a_matrix = np.copy(a_matrix_orig)
# Цикл по столбикам (критериям)
for j, cr in enumerate(criterions):

    # Сумма значений в столбике
    summ_j = sum(a_matrix[j])

    # Цикл по строчкам (значения критериев альтернатив)
    for i, al in enumerate(alternatives):
        a_matrix[j][i] = round(a_matrix[j][i] / summ_j, okr)

# Вывод нормализованной таблицы
fig_norm = Figure(data=[
    Table(
        header=dict(values=['Альтернативы\Критерии'] + criterions),
        cells=dict(
            values=[
                alternatives,
                a_matrix[0],
                a_matrix[1],
                a_matrix[2],
                a_matrix[3]
            ]),
    )])
iplot(fig_norm)

print(weights)
# Экспертная оценка критериев по методу попарного сравнения
gama_matrix = []
alpha_vector = []
for i, w1 in enumerate(weights):
    row = []
    for j, w2 in enumerate(weights):
        if i == j or w2 > w1:
            row.append(0)
        elif w1 > w2:
            row.append(1)
        elif w1 == w2:
            row.append(0.5)

    gama_matrix.append(row)
    alpha_vector.append(sum(row))
print(np.array(gama_matrix))
print(np.array(alpha_vector))
alpha_vector = np.array(alpha_vector) / sum(alpha_vector)
print(alpha_vector)

unit_cr = np.dot(a_matrix, alpha_vector)
# Получение вектора весов критериев

# Объединенный критерий для всех альтернатив
print(unit_cr)
answer = alternatives[np.argmin(unit_cr)]
print(answer)

[0.8, 0.8, 0.2, 0.4]
[[0.  0.5 1.  1. ]
 [0.5 0.  1.  1. ]
 [0.  0.  0.  0. ]
 [0.  0.  1.  0. ]]
[2.5 2.5 0.  1. ]
[0.41666667 0.41666667 0.         0.16666667]
[0.20416667 0.33333083 0.19270833 0.3377175 ]
Грунтовка


Метод анализа иерархий

In [210]:
# Матрица для критерия "Расстояние"
pair_compare_distance = [
    [1,1/2,1/4,1/5],
    [2,1,1/3,1/4],
    [4,3,1,1/2],
    [5,4,2,1]
]
# Матрица для критерия "Качество покрытия"
pair_compare_quality = [
    [1,3,6,9],
    [1/3,1,3,5],
    [1/6,1/3,1,2],
    [1/9,1/5,1/2,1]
]
# Матрица для критерия "Контроль"
pair_compare_control = [
    [1,1/2,1/5,1/6],
    [2,1,1/3,1/5],
    [5,3,1,1/2],
    [6,5,2,1]
]
# Матрица для критерия "Инфраструктура"
pair_compare_infrast = [
    [1,1/2,5,7],
    [2,1,7,9],
    [1/5,1/7,1,2],
    [1/7,1/9,1/2,1]
]

# Матрица для критерия «Оценка приоритетов критериев»
pair_compare_criterion = [
    [1,3,5,9],
    [1/3,1,3,5],
    [1/5,1/3,1,2],
    [1/9,1/5,1/2,1]
]

print("Расстояние")
sum_row_distance = np.round(np.sum(pair_compare_distance, axis=0), okr)
sum_col_distance = np.round(np.sum(pair_compare_distance, axis=1), okr)
norm_sum_distance = np.round(sum_col_distance / np.sum(sum_col_distance), okr)
# Отношение согласованности
consistency_distance = (np.dot(np.sum(pair_compare_distance, axis=0), norm_sum_distance) - len(alternatives)) / (len(alternatives)-1)
print(consistency_distance)

print("Качество покрытия")
sum_row_quality = np.round(np.sum(pair_compare_quality, axis=0),okr)
sum_col_quality = np.round(np.sum(pair_compare_quality, axis=1),okr)
norm_sum_quality = np.round(sum_col_quality / np.sum(sum_col_quality),okr)
# Отношение согласованности
consistency_quality = (np.dot(np.sum(pair_compare_quality, axis=0), norm_sum_quality) - len(alternatives)) / (len(alternatives)-1)
print(consistency_quality)

print("Контроль")
sum_row_control = np.round(np.sum(pair_compare_control, axis=0),okr)
sum_col_control = np.round(np.sum(pair_compare_control, axis=1),okr)
norm_sum_control = np.round(sum_col_control / np.sum(sum_col_control),okr)
# Отношение согласованности
consistency_control = (np.dot(np.sum(pair_compare_control, axis=0), norm_sum_control) - len(alternatives)) / (len(alternatives)-1)
print(consistency_control)

print("Инфраструктура")
sum_row_infrast = np.round(np.sum(pair_compare_infrast, axis=0),okr)
sum_col_infrast = np.round(np.sum(pair_compare_infrast, axis=1),okr)
norm_sum_infrast = np.round(sum_col_infrast / np.sum(sum_col_infrast),okr)
# Отношение согласованности
consistency_infrast = (np.dot(np.sum(pair_compare_infrast, axis=0), norm_sum_infrast) - len(alternatives)) / (len(alternatives)-1)
print(consistency_infrast)

print("Оценка приоритетов критериев")
sum_row_criterion = np.round(np.sum(pair_compare_criterion, axis=0),okr)
sum_col_criterion = np.round(np.sum(pair_compare_criterion, axis=1),okr)
norm_sum_criterion = np.round(sum_col_criterion / np.sum(sum_col_criterion),okr)
# Отношение согласованности
consistency_criterion = (np.dot(np.sum(pair_compare_criterion, axis=0), norm_sum_criterion) - len(alternatives)) / (len(alternatives)-1)
print(consistency_criterion)

matr = np.array([norm_sum_distance, norm_sum_quality, norm_sum_control, norm_sum_infrast])
print(matr)
answer_vector = np.dot(matr, norm_sum_criterion)
print(answer_vector)
answer = alternatives[np.argmax(answer_vector)]
print(answer)

# Вывод таблиц
fig = make_subplots(
    rows=5, cols=1,
    specs=[[{"type": "table"}],
           [{"type": "table"}],
           [{"type": "table"}],
           [{"type": "table"}],
           [{"type": "table"}]]
)
fig.add_trace(
    Table(
        header=dict(
            values=[''] + alternatives + ['Сумма по строке', 'Нормированная сумма по строке']),
        cells=dict(
            values=np.append((np.append([alternatives], pair_compare_distance, axis=0)), [sum_row_distance, norm_sum_distance], axis=0))
    ),
    row=1, col=1
)
fig.add_trace(
    Table(
        header=dict(
            values=[''] + alternatives + ['Сумма по строке', 'Нормированная сумма по строке']),
        cells=dict(
            values=np.append((np.append([alternatives], pair_compare_quality, axis=0)), [sum_row_quality, norm_sum_quality], axis=0))
    ),
    row=2, col=1
)
fig.add_trace(
    Table(
        header=dict(
            values=[''] + alternatives + ['Сумма по строке', 'Нормированная сумма по строке']),
        cells=dict(
            values=np.append((np.append([alternatives], pair_compare_control, axis=0)), [sum_row_control, norm_sum_control], axis=0))
    ),
    row=3, col=1
)
fig.add_trace(
    Table(
        header=dict(
            values=[''] + alternatives + ['Сумма по строке', 'Нормированная сумма по строке']),
        cells=dict(
            values=np.append((np.append([alternatives], pair_compare_infrast, axis=0)), [sum_row_infrast, norm_sum_infrast], axis=0))
    ),
    row=4, col=1
)
fig.add_trace(
    Table(
        header=dict(
            values=[''] + alternatives + ['Сумма по строке', 'Нормированная сумма по строке']),
        cells=dict(
            values=np.append((np.append([alternatives], pair_compare_criterion, axis=0)), [norm_sum_criterion, norm_sum_criterion], axis=0))
    ),
    row=5, col=1
)


fig.update_layout(
    height=1000)
fig.show()

Расстояние
0.04585027777777769
Качество покрытия
0.05828659259259261
Контроль
0.04382555555555564
Инфраструктура
0.05778003703703701
Оценка приоритетов критериев
0.05666500000000004
[[0.0749  0.13764 0.3265  0.46095]
 [0.56473 0.27741 0.10403 0.05383]
 [0.06459 0.12226 0.32872 0.48443]
 [0.35907 0.50536 0.08891 0.04665]]
[0.1414202  0.40453609 0.13288962 0.35432663]
Шоссе
